In [1]:
import json
import os
import collections
import matplotlib.pyplot as plt
import cv2
import numpy as np
from time import sleep
import pandas as pd
from sklearn.model_selection import train_test_split
import shutil

path_name = './data_raw/'
files = os.listdir(path_name)
labels = collections.Counter()
empty_images = 0

labels_dict = {}
all_data = []

with open('./new_data/classes.txt', 'r') as f:
    for i, label in enumerate(f):
        labels_dict[label.strip()] = i
        
images = 0
for file in files:
    if file[-5:] == '.json':
        with open(os.path.join(path_name, file), 'r') as f:
            input_abs_image_path = os.path.join(path_name, file[:-5] + '.jpg')
            data = json.load(f)
            if len(data['shapes']) == 0:
                empty_images += 1
            for object in data['shapes']:
                label = labels_dict[object['label'].replace(' ', '_')]
                bbox_coord = np.array(object['points'])
                if len(bbox_coord) < 4:
                    continue # Потому что там разметка прям совсем кривая
                x_max, y_max = bbox_coord.max(0)
                x_min, y_min = bbox_coord.min(0)

                # assert False
                labels[label] += 1
                all_data.append([str(input_abs_image_path), float(x_min), float(y_min), float(x_max), float(y_max), int(label)])
            
                # break
        # break
                
print(f'Overall {sum(labels.values())} labeled ground truth bboxes.')
print(f'Overall {len(labels)} classes contain this dataset.')  
print(f'Overall {empty_images} no annotated images.')  
all_data = np.array(all_data)

Overall 65596 labeled ground truth bboxes.
Overall 750 classes contain this dataset.
Overall 8 no annotated images.


In [1]:
data=[]
with open('new_data/classes.txt', 'r') as f:
    for line in f:
        data.append(line.strip())
with open('new_data/classes.names', 'w') as f:
    for line in data:
        f.write(line.strip()+'\n')

In [2]:
X = all_data[:,:-1]
y = all_data[:,-1]

upsample_needed_labels = np.unique(y, return_counts=True)[0][np.unique(y, return_counts=True)[1] < 3]
upsample_needed_labels = dict(zip(upsample_needed_labels, np.zeros(upsample_needed_labels.shape)))

upsample_samples = []
for sample in all_data:
    if sample[-1] in upsample_needed_labels:
        if upsample_needed_labels[sample[-1]] < 3:
            upsample_samples.append(sample)
            upsample_needed_labels[sample[-1]] += 1

joined_upsample_and_all_data = np.concatenate((all_data, upsample_samples), axis=0)

In [3]:
print("Итого ббоксов после апсэмпла: ", len(joined_upsample_and_all_data))

Итого ббоксов после апсэмпла:  65739


In [4]:
X_train, X_test, y_train, y_test = train_test_split(joined_upsample_and_all_data[:,:-1], joined_upsample_and_all_data[:,-1], test_size=0.5,                                                                     random_state=42, stratify=joined_upsample_and_all_data[:,-1])

In [5]:
print(X_train.shape, y_train.shape, len(np.unique(y_train)), len(np.unique(y_test)))
train_data = np.concatenate((X_train, y_train.reshape(-1,1)), axis=1)
val_data = np.concatenate((X_test, y_test.reshape(-1,1)), axis=1)
print('Суммарное кол-во ббоксов в трэйне: ', train_data.shape, "Суммарное кол-во ббоксов в тесте: ", val_data.shape, 'Сумма: ', train_data.shape[0] + val_data.shape[0])

(32869, 5) (32869,) 750 750
Суммарное кол-во ббоксов в трэйне:  (32869, 6) Суммарное кол-во ббоксов в тесте:  (32870, 6) Сумма:  65739


In [6]:
merge_train_data = {} # collections.defaultdict(list)
merge_val_data = {} # collections.defaultdict(list)

for sample in train_data:
    if sample[0] not in merge_train_data:
        merge_train_data[sample[0]] = []
    merge_train_data[sample[0]].append(sample[1:])

for sample in val_data:
    if sample[0] not in merge_val_data:
        merge_val_data[sample[0]] = []
    merge_val_data[sample[0]].append(sample[1:])


In [7]:
counter = 0
del_keys = []
for key in merge_val_data.keys():
    if key in merge_train_data:
        del_keys.append(key)
        infos = merge_val_data[key]
        for info in infos:
            a = sum([True if all(info == elem) else False for elem in merge_train_data[key]])
            if not a:
                merge_train_data[key].append(info)
        counter += 1
for key in del_keys:
    del merge_val_data[key]
    
print("Было перемещено ббоксы изображений из вал в трэйн" , counter)

Было перемещено ббоксы изображений из вал в трэйн 11919


In [9]:
print("Итого картинок: ", len(merge_val_data) + len(merge_train_data)) # 14715
counter_train, counter_val = 0,0

counter_target = []
for value in merge_train_data.values():
    counter_train += len(value)
    counter_target.append([i[-1] for i in value])
print("Итого меток в трэйне: ", len(np.unique(np.concatenate(counter_target))))

counter_target = []
for value in merge_val_data.values():
    counter_val += len(value)
    counter_target.append([i[-1] for i in value])
print("Итого меток в тесте: ", len(np.unique(np.concatenate(counter_target))))

print("Итого суммарное число ббоксов: ", counter_train + counter_val)
print("Итого ббоксов в валидации: ", counter_val)
print("Итого ббоксов в трэйне: ",counter_train)

Итого картинок:  14715
Итого меток в трэйне:  750
Итого меток в тесте:  449
Итого суммарное число ббоксов:  65613
Итого ббоксов в валидации:  3620
Итого ббоксов в трэйне:  61993


In [36]:
print('Суммарное кол-во картинок в трэйне: ', len(merge_train_data), "Суммарное кол-во картинок в тесте: ", len(merge_val_data))

Суммарное кол-во картинок в трэйне:  13310 Суммарное кол-во картинок в тесте:  1405


In [37]:
def xyxy2darknet(x_min, y_min, x_max, y_max, image_path):
    img = cv2.imread(image_path)
    height, width, _ = img.shape
    x_min, y_min, x_max, y_max = x_min, y_min, x_max, y_max
    w = x_max - x_min
    h = y_max - y_min
    cx = x_min + w/2
    cy = y_min + h/2
    return cx/width, cy/height, w/width, h/height

def create_paths():
    output_path = 'new_data/'
    if not os.path.isdir(output_path):
        os.mkdir(output_path)
    else:
        shutil.rmtree(output_path)
        os.mkdir(output_path)

    if not os.path.isdir(os.path.join(output_path, 'images')):
        os.mkdir(os.path.join(output_path, 'images'))

    if not os.path.isdir(os.path.join(output_path, 'labels')):
        os.mkdir(os.path.join(output_path, 'labels'))

def create_dataset(classes, files, train_test='train'):
    input_path = 'data_raw/'
    output_path = 'new_data/'

    image_folder = os.path.join(output_path, f'images/{train_test}/')
    label_folder = os.path.join(output_path, f'labels/{train_test}/')

    if not os.path.isdir(image_folder):
        os.mkdir(image_folder)
    
    if not os.path.isdir(label_folder):
        os.mkdir(label_folder)
    
    if os.path.isfile(os.path.join(output_path, f'{train_test}.txt')):
        shutil.remove(os.path.join(output_path, f'{train_test}.txt'))

    labels = {}

    with open(os.path.join('./new_data/', 'classes.txt'), 'w') as f:
        for label in classes.keys():
            f.write(label.strip().replace(' ', '_')+'\n')

    with open('./new_data/classes.txt', 'r') as f:
        for i, label in enumerate(f):
            labels[label.strip()] = i
    
    for file in files.keys():
        input_abs_image_path = file
        output_abs_image_path = file.replace('./data_raw/', image_folder)
        output_abs_labels_path = file.replace('./data_raw/', label_folder)[:-4] + '.txt'
        
        bboxes_with_classes = files[file]
        assert len(bboxes_with_classes) != 0
        with open(os.path.join(output_path, f'{train_test}.txt'), 'a') as f:
            f.write(output_abs_image_path + '\n')
        shutil.copy(input_abs_image_path, output_abs_image_path)
        with open(output_abs_labels_path, 'w') as f:
            for bbox_w_class in bboxes_with_classes:
                x_min, y_min, x_max, y_max = list(map(float, bbox_w_class[:-1]))
                label = int(bbox_w_class[-1])
                x, y, w, h = xyxy2darknet(x_min, y_min, x_max, y_max, input_abs_image_path)
                box_info = f'{label} {x} {y} {w} {h}' + '\n'
                f.write(box_info)

                    

create_paths()
create_dataset(labels_dict, merge_train_data, 'train')
create_dataset(labels_dict, merge_val_data, 'val')